In [2]:
import requests
import pandas as pd
import numpy as np
import folium

In [7]:
conda install -c anaconda xlrd --yes

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [6]:
conda install -c anaconda lxml --yes

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [29]:
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [30]:
import lxml
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head(15)

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,NaN,NaN
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [31]:
df_html.dropna(subset=['Borough'], inplace=True)

In [32]:
na_neighbor = df_html[df_html['Neighborhood'].isna()].shape[0]
na_neighbor

# No na value in neighborhood column

0

In [36]:
df_html[df_html['Borough']=="North York"]

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
11,M3B,North York,Don Mills
14,M6B,North York,Glencairn
20,M3C,North York,Don Mills
46,M2H,North York,Hillcrest Village
47,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North"
55,M2J,North York,"Fairview, Henry Farm, Oriole"
56,M3J,North York,"Northwood Park, York University"


In [45]:
df_pc =pd.DataFrame(df_html.groupby(['Postal Code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)]))
df_pc.reset_index(inplace=True)
df_pc.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [46]:
df_html.dropna(subset=['Borough'], inplace=True)

In [47]:
# number of empty cell in Neighborhood column
na_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print(na_neighborhood)

0


In [48]:
df_pc.shape

(103, 3)

In [49]:
# Turn into csv
df_pc.to_csv('Postcodes_Toronto.csv')

In [52]:
url_csv = 'http://cocl.us/Geospatial_data'
df_cocl = pd.read_csv(url_csv)
df_cocl.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [54]:
df_n = pd.read_csv('Postcodes_Toronto.csv',index_col=[0])
df_n.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [62]:
# change column name
df_cocl.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_n.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [63]:
# check result
df_cocl.head(1)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353


In [64]:
df_n.head(1)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"


In [65]:
df_neighbor_cocl = pd.merge(df_n, df_cocl, on='PostalCode')
df_neighbor_cocl.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [66]:
df_neighbor_cocl.to_csv('Postcodes_2_Toronto.csv')

In [69]:
dft = pd.read_csv('Postcodes_2_Toronto.csv', index_col=0)
dft.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [77]:
# Number of Borough
print(len(dft['Borough'].unique()))

# Number of Neighborhood
print(len(dft['Neighborhood'].unique()))

# get shape
print(dft.shape)

10
99
(103, 5)


In [78]:
dft.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [79]:
df_toronto = dft[dft['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

C:\Users\aa236\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [80]:
boroughs = df_toronto['Borough'].unique().tolist()

In [85]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [83]:
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [86]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [94]:
CLIENT_ID = 'DVE3EJNJJMNU5P5C0IVVMCGET1TOQYBZUE5IPNXKQWNYVWFC' # your Foursquare ID
CLIENT_SECRET = '3P2FEUTUEYM2E0OBDULGVQPB5DPFT4LQVRDML435FHCEXETX' # your Foursquare Secret
VERSION = '20200530' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [95]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [97]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [98]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,62,62,62,62,62,62
Christie,17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [100]:
# Check if Japanese restaurant are in there
"Japanese Restaurant" in toronto_venues['Venue Category'].unique()

True

In [101]:
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 


fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1616, 237)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 237)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.016129,0.000000,0.000000,0.016129,0.000000,0.000000,0.016129
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012987,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.025974
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [103]:
len(to_grouped[to_grouped["Japanese Restaurant"] > 0])

16

In [104]:
Japaneser = to_grouped[["Neighborhoods","Japanese Restaurant"]]

In [105]:
Japaneser.head(9)

,Neighborhoods,Japanese Restaurant
0,Berczy Park,0.017241
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.032258
5,Christie,0.000000
6,Church and Wellesley,0.064935
7,"Commerce Court, Victoria Hotel",0.030000
8,Davisville,0.000000


In [106]:
# We decided to use k-means clustering 
from sklearn.cluster import KMeans
toclusters = 3

to_clustering = Japaneser.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([0, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0])

In [108]:
to_merged = Japaneser.copy()

to_merged["Cluster Labels"] = kmeans.labels_

In [109]:
to_merged.head()

,Neighborhoods,Japanese Restaurant,Cluster Labels
0,Berczy Park,0.017241,0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,1
2,"Business reply mail Processing Centre, South C...",0.000000,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,1
4,Central Bay Street,0.032258,0


In [111]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)

In [112]:
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1616, 9)


,Neighborhood,Japanese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017241,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.017241,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.017241,0,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.017241,0,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.017241,0,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [113]:
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged.head()

,Neighborhood,Japanese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.017241,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
17,"Kensington Market, Chinatown, Grange Park",0.017241,0,43.653206,-79.400049,Buddha's Vegetarian,43.651904,-79.403312,Vegetarian / Vegan Restaurant
17,"Kensington Market, Chinatown, Grange Park",0.017241,0,43.653206,-79.400049,Livelihood Cafe,43.655821,-79.402629,Café
17,"Kensington Market, Chinatown, Grange Park",0.017241,0,43.653206,-79.400049,Otto's Berlin Döner,43.656387,-79.402788,Doner Restaurant
17,"Kensington Market, Chinatown, Grange Park",0.017241,0,43.653206,-79.400049,Market 707,43.652128,-79.404844,Food Court


In [115]:
map_clusters = folium.Map(location=[lat_toronto, lon_toronto],zoom_start=14)

# Create color sceme
# add markers to the map
markers_colors={}
markers_colors[0] = 'blue'
markers_colors[1] = 'red'
markers_colors[2] = 'black'
markers_colors[3] = 'orange'
markers_colors[4] = 'green'
markers_colors[5] = 'yellow'
for lat, lon, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [116]:
#Cluster 0
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['Venue Category'] == 'Japanese Restaurant') ]

,Neighborhood,Japanese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
17,"Kensington Market, Chinatown, Grange Park",0.017241,0,43.653206,-79.400049,Gushi,43.652258,-79.404884,Japanese Restaurant
23,"Queen's Park, Ontario Provincial Government",0.027778,0,43.662301,-79.389494,Tokyo Grill,43.665085,-79.384707,Japanese Restaurant
19,"Little Portugal, Trinity",0.022727,0,43.647927,-79.419750,Bazara,43.648535,-79.420521,Japanese Restaurant
13,"Garden District, Ryerson",0.030000,0,43.657162,-79.378937,Kinka Izakaya Original,43.660596,-79.378891,Japanese Restaurant
13,"Garden District, Ryerson",0.030000,0,43.657162,-79.378937,KAKA,43.657457,-79.384192,Japanese Restaurant
13,"Garden District, Ryerson",0.030000,0,43.657162,-79.378937,Katsuya,43.659860,-79.378788,Japanese Restaurant
37,"Toronto Dominion Centre, Design Exchange",0.030000,0,43.647177,-79.381576,Chotto Matte,43.646473,-79.378782,Japanese Restaurant
37,"Toronto Dominion Centre, Design Exchange",0.030000,0,43.647177,-79.381576,Ki Modern Japanese + Bar,43.647167,-79.379608,Japanese Restaurant
37,"Toronto Dominion Centre, Design Exchange",0.030000,0,43.647177,-79.381576,Ninki Izakaya,43.650228,-79.384863,Japanese Restaurant
30,"St. James Town, Cabbagetown",0.023810,0,43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant


In [117]:
#Cluster 1
to_merged.loc[(to_merged['Cluster Labels'] ==1) & (to_merged['Venue Category'] == 'Japanese Restaurant') ]

,Neighborhood,Japanese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
29,St. James Town,0.012500,1,43.651494,-79.375418,Gyu-Kaku Japanese BBQ,43.651422,-79.375047,Japanese Restaurant
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,1,43.640816,-79.381752,Miku,43.641374,-79.377531,Japanese Restaurant
25,"Richmond, Adelaide, King",0.010989,1,43.650571,-79.384568,Fune Japanese Restaurant,43.648514,-79.386457,Japanese Restaurant


In [120]:
#Cluster 2

to_merged.loc[(to_merged['Cluster Labels'] ==2) & (to_merged['Venue Category'] == 'Japanese Restaurant') ]

,Neighborhood,Japanese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,Church and Wellesley,0.064935,2,43.665860,-79.383160,Kokoni Izakaya,43.664181,-79.380258,Japanese Restaurant
38,"University of Toronto, Harbord",0.058824,2,43.662696,-79.400049,Yasu,43.662837,-79.403217,Japanese Restaurant
38,"University of Toronto, Harbord",0.058824,2,43.662696,-79.400049,Gyubee,43.667088,-79.400571,Japanese Restaurant
6,Church and Wellesley,0.064935,2,43.665860,-79.383160,Tokyo Grill,43.665085,-79.384707,Japanese Restaurant
6,Church and Wellesley,0.064935,2,43.665860,-79.383160,Kawa Sushi,43.663894,-79.380210,Japanese Restaurant
6,Church and Wellesley,0.064935,2,43.665860,-79.383160,Onnki Donburi,43.669757,-79.384574,Japanese Restaurant
6,Church and Wellesley,0.064935,2,43.665860,-79.383160,Tokyo Kitchen,43.668783,-79.385153,Japanese Restaurant


In [121]:
# Cluster 3
to_merged.loc[(to_merged['Cluster Labels'] ==3) & (to_merged['Venue Category'] == 'Japanese Restaurant') ]

,Neighborhood,Japanese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [119]:
# Cluseter 4
to_merged.loc[(to_merged['Cluster Labels'] ==4) & (to_merged['Venue Category'] == 'Japanese Restaurant') ]

,Neighborhood,Japanese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


## As we can see, most Japanese restaurant belongs to cluster 0

There are several Japanese in cluster1 and 2, which are both significant lower than cluster 0, so we can say if we want to open a Japanese restaurant, we should focus on cluster 1 and 2